In [54]:
import pandas as pd
from datetime import datetime

In [55]:
df = pd.read_csv('data/ema/prueba_dia_de_semana.csv', encoding='utf-16', header=1)

In [56]:
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

In [57]:
df['dia'] = df['dia'].str.strip()

In [58]:
df['año'] = df['dia'].str.split(' ').str[4].astype(str)

In [59]:
df['mes'] = df['dia'].str.split(' ').str[2].astype(str)

In [60]:
df['dia_de_la_semana'] = df['dia'].str.split(' ').str[0].astype(str)

In [61]:
mes = 'Enero Febrero Marzo Abril Mayo Junio Julio Agosto Septiembre Octubre Noviembre Diciembre'.split(' ')
num_mes = '1 2 3 4 5 6 7 8 9 10 11 12'.split(' ')
mes_orden = dict(zip(mes, num_mes))

In [62]:
# Genero una columna auxiliar con el numero del mes para generar luego una fecha Parseada
df['mes_numerico'] = df['mes'].map(mes_orden)

In [63]:
df['fecha_parsed'] = df['mes_numerico'].astype(str) + '/' + df['dia_de_la_semana'].astype(str) + '/' + df['año'].astype(str)

In [64]:
# Convierto la fecha parseada a DateTime y asi obtengo el detalle del dia de la semana para luego filtrar la informacion unicamente de los domigos
df['fecha_final'] = pd.to_datetime(df['fecha_parsed'], format='%m/%d/%Y')
# Obtengo el detalle del nombre del dia a partir de la columna generada anteriormente
df['nombre_dia'] = df['fecha_final'].dt.day_name()

In [65]:
# Genero un diccionario Auxiliar para traducir los meses de Ingles a Castellano
dias_de_semana_castellano = {
    'Monday': 'Lunes',
    'Tuesday': 'Martes',
    'Wednesday': 'Miercoles',
    'Thursday': 'Jueves',
    'Friday': 'Viernes',
    'Saturday': 'Sabado',
    'Sunday': 'Domingo'
}
# Genero la columna con los meses en castellano 
df['dia_de_semana_castellano'] = df['nombre_dia'].map(dias_de_semana_castellano)

In [ ]:
if 'punto_operacional' in df.columns:
    df['no'] = df['punto_operacional'].str.split(' ').str[0].astype(int)

    # Cargo y trabajo sobre el PADRON
    # Cargo unicamente las columnas que me van a servir
    cols = ['N°', 'NOMBRE', 'Fecha apertura', 'BANDERA', 'ORGANIZACIÓN ', 'PROVINCIA', 'FIN DE CIERRE', 'ENE.2', 'FEB.2', 'MAR.2', 'ABR.2', 'MAY.2', 'JUN.2', 'JUL.2', 'AGO.2', 'SEP.2', 'OCT.2', 'NOV.2', 'DIC.2']

    mes_comparable = 'octubre'

    # Leo el Padron
    try:
        padron = pd.read_excel('data/tercer semana - briefing/padron.xlsx', header=17, usecols=cols)

    except Exception as e:
        print(f'Error a la hora de cargar el Padron. ERROR: {e}')

    # Estandarizo los nombres de las columnas del padron
    padron.columns = padron.columns.str.lower().str.strip().str.replace(' ', '_').str.replace('.2', '')

    meses_dict = {
    'enero': 'ene',
    'febrero': 'feb',
    'marzo': 'mar',
    'abril': 'abr',
    'mayo': 'may',
    'junio': 'jun',
    'julio': 'jul',
    'agosto': 'ago',
    'septiembre': 'sep',
    'octubre': 'oct',
    'noviembre': 'nov',
    'diciembre': 'dic'
    }

    columna_mes = meses_dict.get(mes_comparable.lower())
    if not columna_mes:
        raise ValueError(f"Mes '{mes_comparable}' no reconocido. Usá un nombre completo (por ejemplo: 'Octubre').")

    # Renombro algunas columnas para que tengan mas sentido
    padron = padron.rename(columns={'n°':'no'})

    # Elimino las filas que tengan NA en su numero, nombre o mes comparable
    padron = padron.dropna(subset=['no', 'nombre', columna_mes], how='any')

    # Convierto la columna de Numero Operacional efectivamente a INT
    padron['no'] = padron['no'].astype(int)

    # Paso a Mayuscula la comparabilidad de superficie
    for col in padron.columns:
        if col in ['ene', 'feb', 'mar', 'abr', 'may', 'jun', 'jul', 'ago', 'sep', 'oct', 'nov', 'dic']:
            padron[col] = padron[col].str.upper()


    # Genero el merge para obtener el detalle de comparabilidad
    df = pd.merge(df, padron[['no', mes_comparable[0:3]]], on='no', how='left')

In [75]:
df.columns = df.columns.str.replace('_', ' ').str.capitalize()

In [ ]:
df.to_csv('results/Prog Vol x Dia de la semana Express.csv', encoding='utf-16', index=False, decimal=',')